In [1]:
import xarray as xr
import numpy as np
import dask.array as da
import matplotlib.pyplot as plt
from xmovie import Movie
import imageio
import os
from dask.distributed import Client, LocalCluster
from datetime import datetime,timedelta
import glob
import indices_function as ifun
import warnings
from rechunker import rechunk

warnings.filterwarnings('ignore')

/home5/skhajehe/.local/lib/python3.10/site-packages/xmovie/core.py:21: UserWarning: Optional dependency `tqdm` not found. This will make progressbars a lot nicer.     Install with `conda install -c conda-forge tqdm`
  warnings.warn(


In [2]:
cluster = LocalCluster(
    n_workers=30, 
    threads_per_worker=1,
    timeout='3600s',
    memory_limit='40GB',   
)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 30
Total threads: 30,Total memory: 1.09 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37865,Workers: 30
Dashboard: http://127.0.0.1:8787/status,Total threads: 30
Started: Just now,Total memory: 1.09 TiB
Comm: tcp://127.0.0.1:34263,Total threads: 1
Dashboard: http://127.0.0.1:34799/status,Memory: 37.25 GiB
Nanny: tcp://127.0.0.1:35223,


In [ ]:
import dask
import xarray as xr

# List of file names
for year in range(1981,2015):
    file_names=glob.glob("/nobackupp10/datapool/NEX-DCP30-CMIP6/*/historical/*/tasmin/*_"+str(year)+".nc")

    # Open each file lazily using Dask and xarray
    datasets = [xr.open_dataset(file).chunk({"time":-1,"lat":800,"lon":800}) for file in file_names]
    def add_time(ds):
        ds_standard= ds.convert_calendar(calendar ='noleap', 
        align_on ='year', missing = np.nan)
        ds_standard['time'] = ds_standard['time'].dt.floor('D')
        return ds_standard
    datasets_with_time= [add_time(ds) for ds in datasets]
    # datasets_with_time = dask.compute(*datasets_with_time_delayed)

    # Concatenate datasets along the time dimension using Dask and xarray
    combined_data = xr.concat(datasets_with_time, dim='files')

    # Calculate the mean across all files using Dask and xarray (lazy computation)
    mean_data = combined_data.mean(dim='files')

    # Save the computed mean data to a new NetCDF file
    # mean_data.to_netcdf('/nobackupp10/skhajehe/dcp-indices/multimodel/tasmin/1981.nc')

    target_chunks = {
        "tasmin": {"time": 365, "lat": 800, "lon": 800},
        "time": None,  # don't rechunk this array
        "lon": None,
        "lat": None,
    }
    max_mem = "15GB"
    os.system("rm -rf /nobackupp10/skhajehe/dcp-indices/multimodel/temp_chunk.zarr")
    os.system("rm -rf /nobackupp10/skhajehe/dcp-indices/multimodel/tasmin/"+str(year)+".zarr")
    target_store = "/nobackupp10/skhajehe/dcp-indices/multimodel/tasmin/"+str(year)+".zarr"
    temp_store = "/nobackupp10/skhajehe/dcp-indices/multimodel/temp_chunk.zarr"
    # need to remove the existing stores or it won't work
    # !rm -rf group_rechunked.zarr group_rechunked-tmp.zarr
    array_plan = rechunk(
        mean_data, target_chunks, max_mem, target_store, temp_store=temp_store
    )
    array_plan.execute()
